# Documents DB File

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"
file_name = "../data/raw/fintech-scopus.csv"
import_scopus_file(file_name, directory)

# !cat ../data/processed/debug/filter.yaml

# Thesaurus

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"

# clean_institutions(directory)
# clean_keywords(directory)
# apply_thesaurus(directory, thesaurus_file="keywords.txt", input_column="author_keywords", output_column="author_keywords_thesaurus", strict=False)
# create_thesaurus(directory, column="keywords", thesaurus_file="test_thesaurus.txt", sep="; ")

# Basic analysis

In [1]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"

# core_authors(directory)
# core_sources(directory)
# coverage(directory)
# impact_analysis(directory, column="authors")
# summary(directory)
# count_documents_by_term(directory, column='authors', min_occ=6)
print(terms_analysis(directory, column='authors', min_occ=5))

terms_per_year_analysis(directory, column='authors', metric='num_documents', min_occ=5)
# time_analysis(directory)
# top_documents(directory)

                    num_documents  global_citations  local_citations
authors                                                             
Rabbani MR/0                   10                69               32
Arner DW/0                      9               135               47
Wojcik D/0                      7                49               14
Buckley RP/0                    7               132               45
Reyes-Mercado P/0               7                 0                0
Khan S/0                        6                49               22
Ozili PK/0                      6               151               33
Schwienbacher A/0               6                50               19
Serrano W/0                     6                15                3
Gozman DP/0                     6                26                8
Wonglimpiyarat J/0              6                52               36
Tan B/0                         5               105               56
Zetzsche DA/0                   5 

,,authors,Rabbani MR/0,Arner DW/0,Wojcik D/0,Reyes-Mercado P/0,Buckley RP/0,Wonglimpiyarat J/0,Serrano W/0,Schwienbacher A/0,Ozili PK/0,Khan S/0,Gozman DP/0,Zetzsche DA/0,Tan B/0,Nieves EH/0,Mention A-L/0,Hamdan A/0,Faccia A/0,Baber H/0,Ashta A/0
,,num_docs,10,9,7,7,7,6,6,6,6,6,6,5,5,5,5,5,5,5,5
,,cited_by,69,135,49,0,132,52,15,50,151,49,26,44,105,15,35,18,27,12,9
pub_year,num_docs,cited_by,,,,,,,,,,,,,,,,,,,
2016,28,598,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017,73,1370,0,1,0,0,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0
2018,132,1801,0,1,1,0,1,1,2,1,1,0,2,0,0,0,1,0,0,0,1
2019,226,1493,0,2,1,0,1,2,1,1,1,0,2,1,0,0,1,0,0,1,0
2020,411,1349,6,3,3,0,3,1,2,0,3,4,1,3,3,1,1,3,3,3,0
2021,415,325,4,1,2,7,1,0,1,4,1,2,1,1,1,3,2,2,2,1,4
2022,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


# Basic plots

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"

# bar_chart(series=time_analysis(directory)['num_documents'], darkness=time_analysis(directory)['global_citations'])
# pie_chart(series=time_analysis(directory)['num_documents'], darkness=time_analysis(directory)['global_citations'])
# column_chart(series=time_analysis(directory)['num_documents'], darkness=time_analysis(directory)['global_citations'])
# word_cloud(terms_analysis(directory, column='authors')['num_documents'].head(50), cmap='Reds')
# tree_map(terms_analysis(directory, column='authors')['num_documents'].head(50), cmap='Reds')

# bradford_plot(directory)
# lotka_plot(directory)
# worldmap(directory, metric="num_documents")

# Advanced analysis

In [ ]:
# BigraphAnalyzer(dirpath_or_records="../data/processed/debug", column='countries', by='countries').table_

In [ ]:
import pandas as pd

pd.read_csv("../data/processed/debug/documents.csv").iso_source_name.value_counts()

In [ ]:
pd.read_csv("../data/processed/debug/documents.csv").columns